### Kütüphanelerin Yüklenmesi

In [1]:
import pandas as pd

### Veri Yüklenmesi

In [2]:
data = pd.read_csv("data/clean_data.csv")

In [3]:
data.sample(5)

,Unnamed: 0,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,...,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Year
1432744,1467285,2.01E+12,498875.0,162100.0,-0.581611,51.349332,45,3,2,1,...,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,2,2014
829243,829549,20104100D0210,506490.0,205750.0,-0.459106,51.740298,41,1,2,2,...,949,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lights present and lit,Raining without high winds,Wet/Damp,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2010
149560,149636,2.01E+12,601090.0,142370.0,0.873585,51.145198,46,3,3,1,...,292,None within 50 metres,non-junction pedestrian crossing,Darkness: Street lights present and lit,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2005
399800,399977,200701PY20422,536970.0,169410.0,-0.032068,51.407126,1,3,1,1,...,2015,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2007
635197,635438,200920E030101,405170.0,284510.0,-1.925351,52.458485,20,3,1,2,...,0,None within 50 metres,non-junction pedestrian crossing,Darkness: Street lights present and lit,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2009


### Okunan veri setinden gerekli olan sütunların filtrelenmesi

In [4]:
data_filter = data[["Local_Authority_(Highway)", "Latitude", "Longitude", "Year"]]

In [5]:
len(data_filter["Local_Authority_(Highway)"].unique())

207

In [6]:
data_filter["Year"].unique()

array([2005, 2006, 2007, 2009, 2010, 2011, 2012, 2013, 2014], dtype=int64)

### Filtrelenen verinin yıllara göre ayrılması

In [8]:
data_2005 = data_filter[data_filter["Year"] == 2005]
data_2006 = data_filter[data_filter["Year"] == 2006]
data_2007 = data_filter[data_filter["Year"] == 2007]
data_2009 = data_filter[data_filter["Year"] == 2009]
data_2010 = data_filter[data_filter["Year"] == 2010]
data_2011 = data_filter[data_filter["Year"] == 2011]
data_2012 = data_filter[data_filter["Year"] == 2012]
data_2013 = data_filter[data_filter["Year"] == 2013]
data_2014 = data_filter[data_filter["Year"] == 2014]

In [9]:
data_2005.to_csv("data_2005.csv")
data_2006.to_csv("data_2006.csv")
data_2007.to_csv("data_2007.csv")
data_2009.to_csv("data_2009.csv")
data_2010.to_csv("data_2010.csv")
data_2011.to_csv("data_2011.csv")
data_2012.to_csv("data_2012.csv")
data_2013.to_csv("data_2013.csv")
data_2014.to_csv("data_2014.csv")

### Posta kodlarını District ismine çevirme

#### Posta kodlarının yeni bir dataframe'e alınması

In [10]:
local_auth = data_filter["Local_Authority_(Highway)"]

In [11]:
local_auth.sample(5)

639908    E08000029
501437    E10000012
7553      E09000026
257523    E08000033
754322    E09000017
Name: Local_Authority_(Highway), dtype: object

#### * https://www.data.gov.uk/ üzerinden bulunan district ve posta kodu veri setinin okunması

In [12]:
local_auth_uk = pd.read_csv("data/local_authority_traffic.csv")

In [13]:
local_auth_uk = local_auth_uk[["name", "ONS_code"]]

In [14]:
local_auth_uk_filter = local_auth_uk.drop_duplicates()

#### LEFT JOIN yapabilmek için sütun isimlerine aynı değerlerin verilmesi

In [15]:
local_auth_uk_filter = local_auth_uk_filter.rename(columns={"ONS_code": "Local_Authority_(Highway)"})

#### Series olan verinin Dataframe'e çevrilmesi

In [16]:
local_auth = local_auth.to_frame()

#### LEFT JOIN ON posta kodu

In [17]:
merged = local_auth.merge(local_auth_uk_filter, on="Local_Authority_(Highway)", how="left")

#### * https://www.data.gov.uk/ 'dan alınan veri setinde olmayan bazı posta kodları bulunmakta

##### Sebep: Veri seti 2014'e kadar olduğu için bazı değişiklikler olmuş olabilir.

In [18]:
merged.isnull().sum()

Local_Authority_(Highway)        0
name                         53858
dtype: int64

#### District ismi olmayan posta kodlarının tespiti

In [19]:
merged_x = merged.loc[merged['name'].isna(), 'Local_Authority_(Highway)']

In [20]:
merged_x.unique()

array(['EHEATHROW', 'E06000048', 'E08000020', 'E10000002', 'S12000024',
       'S12000015', 'S12000043', 'S12000009', 'S12000044'], dtype=object)

#### * https://findthatpostcode.uk/ sayfasından bulunan district isimlerini veri setine ekleme

In [23]:
import pandas as pd

data = {
    
    'name': [
        'Heatrow',
        'Northumberland',
        'Gateshead',
        'Buckinghamshire',
        'Perth and Kinross',
        'Fife',
        'Glasgow City',
        'East Dunbartonshire',
        'North Lanarkshire'
    ],
    'Local_Authority_(Highway)': [
        'EHEATHROW',
        'E06000048',
        'E08000020',
        'E10000002',
        'S12000024',
        'S12000015',
        'S12000043',
        'S12000009',
        'S12000044'
    ],
}

df = pd.DataFrame(data)
print(df)


                  name Local_Authority_(Highway)
0              Heatrow                 EHEATHROW
1       Northumberland                 E06000048
2            Gateshead                 E08000020
3      Buckinghamshire                 E10000002
4    Perth and Kinross                 S12000024
5                 Fife                 S12000015
6         Glasgow City                 S12000043
7  East Dunbartonshire                 S12000009
8    North Lanarkshire                 S12000044


In [22]:
local_auth_uk_filter.sample(5)

,name,Local_Authority_(Highway)
60,Rotherham,E08000018
190,Bradford,E08000032
110,Wokingham,E06000041
178,Bournemouth,E06000028
78,West Sussex,E10000032


In [24]:
local_auth_uk_filter_new = pd.concat([local_auth_uk_filter, df])

In [25]:
local_auth_uk_filter_new

,name,Local_Authority_(Highway)
0,Isles of Scilly,E06000053
1,Nottinghamshire,E10000024
2,Glasgow City,S12000049
3,North Lanarkshire,S12000050
4,Somerset,E10000027
...,...,...
4,Perth and Kinross,S12000024
5,Fife,S12000015
6,Glasgow City,S12000043
7,East Dunbartonshire,S12000009


### Posta kodları ve posta kodlarının district isimleriyle birlikte bulunan veri setini ana veri setinin birleştirilmesi

In [26]:
final_merge = local_auth.merge(local_auth_uk_filter_new, on="Local_Authority_(Highway)", how="left")

#### Eksik veri var mı kontrolü

In [28]:
len(final_merge)

1469609

In [29]:
len(data_filter)

1469609

In [30]:
data_filter["District"] = final_merge["name"]

In [31]:
data_filter.to_csv("data_filter.csv")

### TABLEAU LINK

https://public.tableau.com/shared/Y2X6DY3KJ?:display_count=n&:origin=viz_share_link